In [1]:
from django.contrib.gis.db.models.functions import Area
from django.contrib.gis.geos import Polygon
from django.db.models import Case, CharField, DecimalField, F, Q, Sum, Value, When
from django.db.models.functions import Cast, Concat

# from django.db.models import  F, Max, Min, Q, ,   
# from django.db.models.functions import Coalesce,  Cast

from project.models import Project
from public_data.models import OcsgeDiff, Ocsge, ZoneUrba


zone_urba = ZoneUrba.objects.get(pk=599878)
diagnostic = Project.objects.get(pk=1504)
zone_urba, diagnostic

(<ZoneUrba: ZoneUrba object (599878)>,
 <Project: Diagnostic de CA Grand Auch Coeur de Gascogne>)

In [2]:
Zero = Area(Polygon(((0, 0), (0, 0), (0, 0), (0, 0)), srid=2154))
Zero

Area(Value(<Polygon object at 0x7f2fb52eac90>))

In [3]:
(
    OcsgeDiff.objects.intersect(zone_urba.mpoly)
    .filter(year_old__gte=diagnostic.first_year_ocsge, year_new__lte=diagnostic.last_year_ocsge)
    .filter(Q(is_new_artif=True) | Q(is_new_natural=True))
    .annotate(
        period=Cast(Concat("year_old", Value(" - "), "year_new"), CharField(max_length=15)),
        area_artif=Case(When(is_new_artif=True, then=F("intersection_area")), default=Zero),
        area_renat=Case(When(is_new_natural=True, then=F("intersection_area")), default=Zero),
    )
    .values("period")
    .annotate(
        new_artif=Cast(Sum("area_artif") / 10000, DecimalField(max_digits=15, decimal_places=2)),
        new_nat=Cast(Sum("area_renat") / 10000, DecimalField(max_digits=15, decimal_places=2)),
    )
)


<QuerySet [{'period': '2016 - 2019', 'new_artif': Decimal('12.57'), 'new_nat': Decimal('0.00')}]>

In [4]:
(
    Ocsge.objects.intersect(zone_urba.mpoly)
    .filter(is_artificial=True)
    .aggregate(area=Sum("intersection_area") / 10000)
)

{'area': 12.983959962204459}

In [5]:
"""(
    CommuneSol.objects.filter(
        city__in=self.cities.all(),
        year=self.last_year_ocsge,
        matrix__is_artificial=True,
    )
    .annotate(
        code_prefix=F(f"matrix__{sol}__code_prefix"),
        label=F(f"matrix__{sol}__label"),
        label_short=F(f"matrix__{sol}__label_short"),
        map_color=F(f"matrix__{sol}__map_color"),
    )
    .values("code_prefix", "label", "label_short", "map_color")
    .annotate(surface=Sum("surface"))
)
"""
last_year_ocsge = 2019
(
    Ocsge.objects.intersect(zone_urba.mpoly)
    .filter(is_artificial=True, year=last_year_ocsge)
    .annotate(
        code_prefix=F(f"matrix__couverture__code_prefix"),
        label=F(f"matrix__couverture__label"),
        label_short=F(f"matrix__couverture__label_short"),
        map_color=F(f"matrix__couverture__map_color"),
    )
    .order_by("code_prefix", "label", "label_short", "map_color")
    .values("code_prefix", "label", "label_short", "map_color")
    .annotate(surface=Sum("surface"))
)

<QuerySet [{'code_prefix': 'CS1.1.1.1', 'label': 'Zones bâties', 'label_short': 'Zones bâties', 'map_color': '#ff377a', 'surface': Decimal('26193.4009')}, {'code_prefix': 'CS1.1.1.2', 'label': 'Zones non bâties (Routes; places; parking…)', 'label_short': 'Zones non bâties', 'map_color': '#ff9191', 'surface': Decimal('213978.4976')}, {'code_prefix': 'CS2.2.1', 'label': 'Formations herbacées (Pelouses et prairies; terres arables; roselières; …)', 'label_short': 'Formations herbacées', 'map_color': '#ccf24d', 'surface': Decimal('78269.8068')}]>